In [ ]:
import os
import sys
import json
import warnings
import jwt
import argparse
import boto3

from dkube.sdk import *

#### Configuration

In [ ]:
# Tune the following in this cell 

## specify your Dkube username
user = DKUBEUSERNAME = os.getenv("DKUBE_USER_LOGIN_NAME","")

## Dkube information.
TOKEN = os.getenv("DKUBE_USER_ACCESS_TOKEN","")
DKUBE_URL = os.getenv("DKUBE_URL","")

if not(TOKEN or DKUBEUSERNAME or DKUBE_URL):
    print("Please fill the Dkube details first (TOKEN, DKUBE_URL, DKUBEUSERNAME, DKUBE_IP)")
    raise TypeError

In [ ]:
# Read inputs
with open("inputs.json") as f:
    teamb_inputs = json.load(f)
    

s3_client = boto3.client('s3', 
                        endpoint_url = teamb_inputs["s3_endpoint"],
                        aws_access_key_id=teamb_inputs['s3_accesskeyid'],
                        aws_secret_access_key=teamb_inputs['s3_accesskey'])
    
artifacts_data = s3_client.get_object(Bucket=teamb_inputs["s3_bucket"], 
                                Key=teamb_inputs["artifacts_json"]) \
                .get('Body').read().decode("utf-8")


### Import for Deployment

In [ ]:
api = DkubeApi(URL=DKUBE_URL, token=TOKEN)

artifacts = json.loads(artifacts_data)
deploy_exports = artifacts["deployment_inputs"]
monitor_exports = artifacts["monitor_inputs"]

#1. download model
name = teamb_inputs["name"]

model = DkubeModel(user, name=name + "-model")
model.update_s3_details(endpoint=teamb_inputs["s3_endpoint"], 
                        bucket=teamb_inputs["s3_bucket"], 
                        prefix=deploy_exports["model"], 
                        key=teamb_inputs["s3_accesskeyid"], 
                        secret=teamb_inputs["s3_accesskey"])
api.create_model(model)
    



#2. create transformer code
transformer = deploy_exports["transformer"]
if transformer["enable"] == True:
    code = DkubeCode(user, name=name + "-transformer")
    code.update_git_details(transformer["code"]["url"], branch=transformer["code"]["branch"])
    api.create_code(code)

#4. publish model
publish = DkubeServing(user, name="publish-"+name, description='model published from siteA')
publish.update_serving_model(name + "-model")
publish.update_serving_image(image_url=deploy_exports["predictor"]["image"])
if transformer["enable"] == True:
    publish.set_transformer(True, script=transformer["script"])
    publish.update_transformer_code(code.name)
    publish.update_transformer_image(image_url=transformer["image"])

api.publish_model("publish-"+name, "model published from siteA", publish)

puburl = "/#/ds/repos/models/published/user/{}/{}".format(user, name+"-model")
print("\n")
print("\033[1;31m Imported Resource Details Below  \033[0m \n")
print("\033[1;30m \t Imported model - %s \033[0m " % (name+"-model"))
print("\033[1;30m \t Transformer code - %s \033[0m " % (name+"-transformer"))
print("\033[1;30m \t Published model link - %s \033[0m " % puburl)